In [7]:
import pickle
import json
import numpy as np
import torch
import clip

model, preprocess = clip.load('RN50', device='cuda')

In [8]:
with open('vg/vg_num_13936_len_9.pkl', 'rb') as file:
    vg = pickle.load(file)
vg_concepts = list(vg.keys())

with open('vg/cav_concepts.json', 'r') as file:
    cav = json.load(file)
cav_concepts = cav['concepts']

# 1. cifar 10

In [9]:
from torchvision.datasets import CIFAR10
dataset = CIFAR10(root='/data/.../', train=True, download=False)
classes = list(dataset.classes)

with open('cifar10/cifar10_pcbm_concepts_175.pkl', 'rb') as file:
    pcbm = pickle.load(file)
pcbm_concepts = list(pcbm.keys())

concepts = []

for cpt in cav_concepts:
    if cpt in vg_concepts and cpt not in classes:
        concepts.append(cpt)
for cpt in pcbm_concepts:
    if cpt in vg_concepts and cpt not in classes:
        concepts.append(cpt)

In [10]:
concept_bank = {}
concept_length = 0

with torch.no_grad():
    for cpt in concepts:
        text = clip.tokenize(cpt).cuda()
        text_features = model.encode_text(text).cpu().numpy()
        text_features = text_features / np.linalg.norm(text_features)
        concept_bank[cpt] = text_features[0]
        concept_length += len(cpt)

concept_length = int(concept_length / len(concepts)) + 1

In [11]:
with open(f'cifar10/cifar10_res_num_{len(concept_bank)}_len_{concept_length}.pkl', 'wb') as file:
    pickle.dump(concept_bank, file)

# 2. CIFAR 100

In [12]:
from torchvision.datasets import CIFAR100
dataset = CIFAR100(root='/data/scm22/', train=True, download=False)
classes = list(dataset.classes)

with open('cifar100/cifar100_pcbm_concepts_440.pkl', 'rb') as file:
    pcbm = pickle.load(file)
pcbm_concepts = list(pcbm.keys())

concepts = []

for cpt in cav_concepts:
    if cpt in vg_concepts and cpt not in classes:
        concepts.append(cpt)
for cpt in pcbm_concepts:
    if cpt in vg_concepts and cpt not in classes:
        concepts.append(cpt)

In [13]:
concept_bank = {}
concept_length = 0

with torch.no_grad():
    for cpt in concepts:
        text = clip.tokenize(cpt).cuda()
        text_features = model.encode_text(text).cpu().numpy()
        text_features = text_features / np.linalg.norm(text_features)
        concept_bank[cpt] = text_features[0]
        concept_length += len(cpt)

concept_length = int(concept_length / len(concepts)) + 1

In [14]:
with open(f'cifar100/cifar100_res_num_{len(concept_bank)}_len_{concept_length}.pkl', 'wb') as file:
    pickle.dump(concept_bank, file)